In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.2 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=4d66969b8433a4192ec899ba1395409ee2d02020b8f5b636649088036a3cb120
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [90]:
!pip install bert-tensorflow

In [91]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [92]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

ln: failed to create symbolic link '/mydrive/MyDrive': File exists
'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [93]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

[Errno 2] No such file or directory: 'gdrive/MyDrive/2021-online-language-offline-impact-main'
/content/gdrive/My Drive/2021-online-language-offline-impact-main


In [94]:
%cd ..

/content/gdrive/MyDrive


In [95]:
!ls

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [96]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [97]:
ct_path_601 = 'csv/US06_05.csv'

In [98]:
ct = pd.read_csv(ct_path_601)

FileNotFoundError: ignored

In [ ]:
ct.shape[1]

In [ ]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [ ]:
#len(columns1)
len(columns2)

In [ ]:
ct.columns=columns2

In [ ]:
ct

In [ ]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [ ]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [ ]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [ ]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [ ]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [ ]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [ ]:
ct['NER-msg'] = ct['message'].apply(ner)

In [ ]:
ct['NER-desc'] = ct['description'].apply(ner)

In [ ]:
ct['NER-title'] = ct['title'].apply(ner)

In [ ]:
ct[['NER-msg', 'NER-desc']].head()

In [ ]:
ct.to_csv('a0605.csv')

In [ ]:
ct=pd.read_csv('a0605.csv')

In [ ]:
!ls

In [ ]:
###
#ct=pd.read_csv('a0601.csv')

In [ ]:
ct

In [ ]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [ ]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [ ]:
###
ct_ner = ct

In [ ]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

In [ ]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [ ]:
ct_ner['NER-msg']

In [ ]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [ ]:
ct_ner['NER']

In [ ]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [ ]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [ ]:
ct_ner

In [ ]:
ct_ner['NER-1']=ct_ner['NER']

In [ ]:
ct_ner['NER-1'][0]

In [ ]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
ct_ner[:100]

In [ ]:
%cd mapping_US

In [ ]:
!ls

In [ ]:
ct_mapping=pd.read_csv('dataframe_mapping_0605.csv',encoding='ISO-8859-1')

In [ ]:
ct_mapping

In [ ]:
country_list=pd.read_csv('Country.txt')
country_list['Name']=country_list['Name'].str.lower()

In [ ]:
country_list['Name']

In [ ]:
states_list=pd.read_csv('states.csv')
states_list['State']

In [ ]:
from numpy.core.numeric import NaN
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['City'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

In [ ]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(states_list['State'])):
    if ct_mapping['Mapping'][i]==states_list['State'][k]:
      ct_mapping['Mapping'][i]=NaN

In [ ]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['Mapping'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
ct_mapping

In [ ]:
ct_mapping

In [ ]:
ct_mapping.to_csv('dataframe_mapping_0605.csv')

In [ ]:
#ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [ ]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

In [ ]:
ct_ner[:100]

In [ ]:
ct_ner['NER-1'][:100]

In [ ]:
ct_ner.to_csv('dataframe0605_post_remove.csv')

In [ ]:
!ls